<a href="https://colab.research.google.com/github/mannixp/PDF_Equation/blob/main/Diffusion_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1D Diffusion**

This notebook validates the Fokker-Planck like equation derived in the paper Craske J. et al. 2024 by computing each of the relevant terms numerically for the case of 1D scalar diffusion on a bounded domain. To solve this equation we use the open source pseudo-spectral code Dedalus.

**Setup**

This cell checks if Dedalus is installed and performs some other basic setup.


In [1]:
# Set environment variables for best performance
%env OMP_NUM_THREADS=1
%env NUMEXPR_MAX_THREADS=1

# Minimize logging output
import logging
logging.disable(logging.DEBUG)

# Check if running on google colab
import os
using_google_colab = bool(os.getenv("COLAB_RELEASE_TAG"))

# Check for Dedalus
try:
    import dedalus.public as de
    print("Dedalus already installed :)")
except:
    print("Dedalus not installed yet.")
    if using_google_colab:
        print("Installing for Google Colab.")
        print()
        # Step 1: Install FFTW
        !apt-get install libfftw3-dev
        !apt-get install libfftw3-mpi-dev
        # Step 2: Set paths for Dedalus installation
        import os
        os.environ['MPI_INCLUDE_PATH'] = "/usr/lib/x86_64-linux-gnu/openmpi/include"
        os.environ['MPI_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
        os.environ['FFTW_INCLUDE_PATH'] = "/usr/include"
        os.environ['FFTW_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
        # Step 3: Install Dedalus using pip
        !pip3 install --no-cache http://github.com/dedalusproject/dedalus/zipball/master/
        !pip3 install -q ipympl
        # Step 4: Check installation
        print()
        try:
            import dedalus.public as de
            print("Dedalus successfully installed :)")
        except:
            print("Error installing Dedalus :(")
            raise
    else:
        print("See website for installation instructions:")
        print("https://dedalus-project.readthedocs.io/en/latest/pages/installation.html")

# Setup interactive matplotlib
if using_google_colab:
    from google.colab import output
    output.enable_custom_widget_manager()

env: OMP_NUM_THREADS=1
env: NUMEXPR_MAX_THREADS=1
Dedalus not installed yet.
Installing for Google Colab.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,654 kB of archives.
After this operation, 24.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubuntu8 [770 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu8 [335 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu8 [614 kB]
Get:4 http://archiv


**Content**

First let's import everything we need to run the rest of the notebook.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
import h5py
logger = logging.getLogger(__name__)
%matplotlib widget


Next we solve the equation for the scalar concentration
\begin{equation}
\partial_t C = \kappa \partial_z^2 C, \quad z \in [0,1],
\end{equation}
over the time interval $t \in [0,1]$ subject to the Robin boundary conditions
\begin{aligned}
 C - \beta \partial_z C &= g_0(t), \quad \text{at} \quad z = 0, \\
 C + \beta \partial_z C &= g_1(t), \quad \text{at} \quad z = 1,
\end{aligned}
for the initial condition
\begin{equation}
C(z,t=0) = \tanh(10(z-1/2)).
\end{equation}

In [4]:
# Parameters
β = 2
Nz= 64
kappa = 0.1

stop_sim_time = 1
timestep      = 1e-03

zcoord = d3.Coordinate('z')
dist   = d3.Distributor(zcoord, dtype=np.float64)
zbasis = d3.ChebyshevT(zcoord, size=Nz, bounds=(0,1),dealias=3/2)

# Fields
C      = dist.Field(name='c', bases=zbasis)
tau_c1 = dist.Field(name='tau_c1')
tau_c2 = dist.Field(name='tau_c2')
g0     = dist.Field(name='g0')
g1     = dist.Field(name='g1')

# Substitutions
dz = lambda A: d3.Differentiate(A, zcoord)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
Cz = dz(C)  + lift(tau_c1)
Czz= dz(Cz) + lift(tau_c2)

# Problem
problem = d3.IVP([C, tau_c1, tau_c2], namespace=locals())
problem.add_equation("dt(C) - kappa*Czz = 0")
problem.add_equation("C(z=0) - β*dz(C)(z=0) = g0")
problem.add_equation("C(z=1) + β*dz(C)(z=1) = g1")

# Solver
solver = problem.build_solver(d3.RK222)
solver.stop_sim_time = stop_sim_time

# Initial condition
z      = dist.local_grid(zbasis)
C['g'] = np.tanh(10*(z-.5))

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=100)
flow.add_property(d3.Integrate(C**2) , name='<C^2>')
flow.add_property(d3.Integrate(Cz**2), name='<Cz^2>')

B_video = [];

# Main loop
try:
    logger.info('Starting main loop')
    while solver.proceed:

        # Specify the bcs
        g0['g'] =  np.sin(1.1*solver.sim_time)
        g1['g'] =  np.sin(1.5*solver.sim_time)

        solver.step(timestep)
        if (solver.iteration-1) % 100 == 0:

            C2_avg  = flow.grid_average('<C^2>')
            Cz2_avg = flow.grid_average('<Cz^2>' )

            logger.info('Iteration=%i, Time=%e, dt=%e'%(solver.iteration, solver.sim_time, timestep))
            logger.info('<C^2>=%f, <Cz^2>    =%f'%(C2_avg,Cz2_avg))

        # Capture the last 10 snapshots
        if  solver.iteration == int(stop_sim_time/timestep) - 5:
            snapshots = solver.evaluator.add_file_handler('snapshots', iter=1)
            snapshots.add_task(C , layout='g',name='C' ,scales=3/2)
            snapshots.add_task(Cz, layout='g',name='Cz',scales=3/2)

except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

INFO:subsystems:Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.5e+01/s


2024-01-15 16:41:26,295 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 1.5e+01/s


INFO:__main__:Starting main loop


2024-01-15 16:41:26,302 __main__ 0/1 INFO :: Starting main loop


INFO:__main__:Iteration=1, Time=1.000000e-03, dt=1.000000e-03


...

2024-01-15 16:41:26,952 solvers 0/1 INFO :: Run time (iter 10-end): 0.4729 sec


INFO:solvers:CPU time (iter 10-end): 0.0001314 cpu-hr


2024-01-15 16:41:26,960 solvers 0/1 INFO :: CPU time (iter 10-end): 0.0001314 cpu-hr


INFO:solvers:Speed: 2.763e+05 mode-stages/cpu-sec


2024-01-15 16:41:26,962 solvers 0/1 INFO :: Speed: 2.763e+05 mode-stages/cpu-sec
